1.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

def sdot(s, t, params, Pr):
    M, P = s
    km0, km, K, n, kdm, kp, kdp = params

    rate_M_prod = km0 + km * (K**n / (Pr**n + K**n))
    rate_M_loss = kdm * M

    rate_P_prod = kp * M
    rate_P_loss = kdp * P
    
    dM = rate_M_prod - rate_M_loss
    dP = rate_P_prod - rate_P_loss
    
    return [dM, dP]

# Define parameter values
km0 = 0.01
km = 5
K = 500
n = 2
kdm = 0.1386
kp = 1.2
kdp = 0.0165

params = [km0, km, K, n, kdm, kp, kdp]

# Define initial conditions
M0 = 0
P0 = 0
s0 = [M0, P0]

# Define time observation points
t_start = 0
t_end = 1000
t_obs = np.linspace(t_start, t_end, 1000)

# Scenario 1: Repressor protein Pr is absent (Pr = 0)
Pr0 = 0
s_obs0 = odeint(sdot, s0, t_obs, args=(params, Pr0))

# Scenario 2: Repressor protein Pr is at high levels (Pr = 3000)
Pr3000 = 3000
s_obs3000 = odeint(sdot, s0, t_obs, args=(params, Pr3000))

# Plotting results
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(t_obs, s_obs0[:,0], label='mRNA Level (Pr=0)')
plt.plot(t_obs, s_obs0[:,1], label='Protein Level (Pr=0)')
plt.title('Protein Expression with Pr = 0')
plt.xlabel('Time (minutes)')
plt.ylabel('Concentration')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(t_obs, s_obs3000[:,0], label='mRNA Level (Pr=3000)')
plt.plot(t_obs, s_obs3000[:,1], label='Protein Level (Pr=3000)')
plt.title('Protein Expression with Pr = 3000')
plt.xlabel('Time (minutes)')
plt.ylabel('Concentration')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

def sdot(s, t, params, Pr):
    M, P = s
    km0, km, K, n, kdm, kp, kdp = params

    rate_M_prod = km0 + km * (K**n / (Pr**n + K**n))
    rate_M_loss = kdm * M

    rate_P_prod = kp * M
    rate_P_loss = kdp * P
    
    dM = rate_M_prod - rate_M_loss
    dP = rate_P_prod - rate_P_loss
    
    return [dM, dP]

# Define parameter values
km0 = 0.01
km = 5
K = 500
kdm = 0.1386
kp = 1.2
kdp = 0.0165

# Define Pr values to simulate
Pr_vals = np.arange(0, 5000, 10)

# Define time observation points
t_start = 0
t_end = 1000
t_obs = np.linspace(t_start, t_end, 100)

# Simulate for different values of n and collect steady state protein levels
ns = [1, 2, 4]
results = {}

for n in ns:
    P_ss_vals = []
    params = [km0, km, K, n, kdm, kp, kdp]
    for Pr in Pr_vals:
        s0 = [0, 0]  # Initial conditions for M and P
        s_obs = odeint(sdot, s0, t_obs, args=(params, Pr))
        P_ss_vals.append(s_obs[-1, 1])  # Take the last value as steady state
    results[n] = P_ss_vals

# Plotting results
fig, ax = plt.subplots(figsize=(10, 6))
for n, P_ss_vals in results.items():
    ax.plot(Pr_vals, P_ss_vals, label=f'n={n}')

ax.set_xlabel('Repressor Protein Level (Pr)')
ax.set_ylabel('Steady State Protein Level (P_ss)')
ax.set_title('Protein Steady State Levels under Varying Repressor Levels and Cooperativity')
ax.legend()
plt.show()


2i.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

def sdot2(s, t, params):
    ML, PL, MT, PT = s
    km0L, kmL, KL, nL, kdmL, kpL, kdpL, km0T, kmT, KT, nT, kdmT, kpT, kdpT = params

    rate_ML_prod = km0L + kmL * (KT**nT / (PT**nT + KT**nT))
    rate_ML_loss = kdmL * ML

    rate_PL_prod =  kpL * ML
    rate_PL_loss =  kdpL * PL

    rate_MT_prod = km0T + kmT * (KL**nL / (PL**nL + KL**nL))
    rate_MT_loss = kdmT * MT

    rate_PT_prod = kpT * MT
    rate_PT_loss = kdpT * PT
    
    dML = rate_ML_prod - rate_ML_loss
    dPL = rate_PL_prod - rate_PL_loss
    dMT = rate_MT_prod - rate_MT_loss
    dPT = rate_PT_prod - rate_PT_loss
    
    return [dML, dPL, dMT, dPT]

# define parameter values
km0L = 0.01
kmL = 5
KL = 500
nL = 2
kdmL = 0.1386
kpL = 1.2
kdpL = 0.0165

km0T = 0.01
kmT = 5
KT = 500
nT = 2
kdmT = 0.1386
kpT = 1.2
kdpT = 0.0165

params = [km0L, kmL, KL, nL, kdmL, kpL, kdpL, km0T, kmT, KT, nT, kdmT, kpT, kdpT]

# Define two scenarios for initial conditions
initial_conditions = [
    [0, 0, 36, 2600],  # Starting with TetR expression close to its unrepressed level and no initial expression of LacI
    [36, 2600, 0, 0]   # Starting with LacI expression close to its unrepressed level and no initial expression of TetR
]

t_start = 0
t_end = 1000
t_obs = np.arange(t_start, t_end+0.1, 1)

# Simulate both scenarios
results = []
for s0 in initial_conditions:
    s_obs = odeint(sdot2, s0, t_obs, args=(params,))
    results.append(s_obs)

# Plotting results
plt.figure(figsize=(12, 6))
labels = ['TetR high, LacI low', 'LacI high, TetR low']
for i, result in enumerate(results):
    plt.subplot(1, 2, i + 1)
    plt.plot(t_obs, result[:, 1], label='LacI Protein')
    plt.plot(t_obs, result[:, 3], label='TetR Protein')
    plt.title(labels[i])
    plt.xlabel('Time')
    plt.ylabel('Protein Count')
    plt.legend()

plt.tight_layout()
plt.show()


2ii)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

def sdot_n1(s, t, params, n):
    ML, PL, MT, PT = s
    km0L, kmL, KL, kdmL, kpL, kdpL, km0T, kmT, KT, kdmT, kpT, kdpT = params

    rate_ML_prod = km0L + kmL * (KT**n / (PT**n + KT**n))
    rate_ML_loss = kdmL * ML

    rate_PL_prod = kpL * ML
    rate_PL_loss = kdpL * PL

    rate_MT_prod = km0T + kmT * (KL**n / (PL**n + KL**n))
    rate_MT_loss = kdmT * MT

    rate_PT_prod = kpT * MT
    rate_PT_loss = kdpT * PT
    
    return [rate_ML_prod - rate_ML_loss, rate_PL_prod - rate_PL_loss, rate_MT_prod - rate_MT_loss, rate_PT_prod - rate_PT_loss]

# Parameter values
params = [0.01, 5, 500, 0.1386, 1.2, 0.0165, 0.01, 5, 500, 0.1386, 1.2, 0.0165]
n = 1  # Cooperativity factor

# Initial conditions for two scenarios
initial_conditions = [
    [0, 0, 36, 2600],  # TetR high, LacI low
    [36, 2600, 0, 0]   # LacI high, TetR low
]

t_obs = np.linspace(0, 1000, 1000)  # Observation times

# Run simulation for both scenarios
results_n1 = []
for s0 in initial_conditions:
    result = odeint(sdot_n1, s0, t_obs, args=(params, n))
    results_n1.append(result)

# Plot results for n=1
plt.figure(figsize=(10, 5))
plt.suptitle("Toggle Switch System Behavior at n=1")
for i, result in enumerate(results_n1):
    plt.subplot(1, 2, i+1)
    plt.plot(t_obs, result[:, 1], label='LacI Protein')
    plt.plot(t_obs, result[:, 3], label='TetR Protein')
    plt.title(['TetR high, LacI low', 'LacI high, TetR low'][i])
    plt.xlabel('Time (minutes)')
    plt.ylabel('Protein count')
    plt.legend()
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()


In [ ]:
n_vals = np.arange(1, 4.1, 0.1)
PL_final_1 = []
PL_final_2 = []

for n in n_vals:
    final_results = []
    for s0 in initial_conditions:
        result = odeint(sdot_n1, s0, np.linspace(0, 10000, 10000), args=(params, n))
        final_results.append(result[-1, 1])
    PL_final_1.append(final_results[0])
    PL_final_2.append(final_results[1])

# Plot bifurcation plot
plt.figure(figsize=(10, 5))
plt.plot(n_vals, PL_final_1, label='Final PL (TetR high initial)')
plt.plot(n_vals, PL_final_2, label='Final PL (LacI high initial)')
plt.xlabel('n value')
plt.ylabel('Final LacI Protein Level (PL)')
plt.title('Bifurcation Plot of Toggle Switch System')
plt.legend()
plt.show()


2iii)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

def sdot3(s, t, params):
    PL, PT = s
    vp0L, vpL, vdpL, KL, nL, vp0T, vpT, vdpT, KT, nT = params

    rate_PL_prod = vp0L + vpL * (KT**nT / (PT**nT + KT**nT))
    rate_PL_loss = vdpL * PL

    rate_PT_prod = vp0T + vpT * (KL**nL / (PL**nL + KL**nL))
    rate_PT_loss = vdpT * PT
    
    dPL = rate_PL_prod - rate_PL_loss
    dPT = rate_PT_prod - rate_PT_loss
    
    return [dPL, dPT]

# Define parameter values
vp0L = 0.0866
vpL = 43.3
vdpL = 0.0165
KL = 500
nL = 2

vp0T = 0.0866
vpT = 43.3
vdpT = 0.0165
KT = 500
nT = 2

params = [vp0L, vpL, vdpL, KL, nL, vp0T, vpT, vdpT, KT, nT]

# Initial conditions
s0_vals = [
    [1000, 2600],  # PL, PT
    [2600, 1000],
    [500, 200],
    [200, 500]
]

t_obs = np.linspace(0, 1000, 1000)  # Observation times

# Plotting
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(1, 1, 1)

for s0 in s0_vals:
    s_obs = odeint(sdot3, s0, t_obs, args=(params,))
    PL_obs = s_obs[:, 0]
    PT_obs = s_obs[:, 1]
    ax.plot(PL_obs, PT_obs, label=f'Initial PL={s0[0]}, PT={s0[1]}')

# Nullclines
PT_vals = np.linspace(0, 3000, 300)
PL_nullcline = [((vp0L / vdpL) + (vpL / vdpL) * (KT**nT / (pt**nT + KT**nT))) for pt in PT_vals]
PT_nullcline = [((vp0T / vdpT) + (vpT / vdpT) * (KL**nL / (pl**nL + KL**nL))) for pl in PL_nullcline]

ax.plot(PL_nullcline, PT_vals, 'r--', label='PL Nullcline')
ax.plot(PT_nullcline, PL_nullcline, 'b--', label='PT Nullcline')

ax.set_xlim([0, 3000])
ax.set_ylim([0, 3000])
ax.set_xlabel('LacI Protein (PL)')
ax.set_ylabel('TetR Protein (PT)')
ax.set_title('Phase Plot of Toggle Switch System')
ax.legend()

plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

def sdot3(s, t, params):
    PL, PT = s
    vp0L, vpL, vdpL, KL, nL, vp0T, vpT, vdpT, KT, nT = params

    rate_PL_prod = vp0L + vpL * (KT**nT / (PT**nT + KT**nT))
    rate_PL_loss = vdpL * PL

    rate_PT_prod = vp0T + vpT * (KL**nL / (PL**nL + KL**nL))
    rate_PT_loss = vdpT * PT
    
    return [rate_PL_prod - rate_PL_loss, rate_PT_prod - rate_PT_loss]

# Define parameter values with nL and nT set to 1
vp0L = 0.0866
vpL = 43.3
vdpL = 0.0165
KL = 500
nL = 1  # Reduced cooperativity

vp0T = 0.0866
vpT = 43.3
vdpT = 0.0165
KT = 500
nT = 1  # Reduced cooperativity

params = [vp0L, vpL, vdpL, KL, nL, vp0T, vpT, vdpT, KT, nT]

# Initial conditions
s0_vals = [
    [1000, 2600],  # PL, PT
    [2600, 1000],
    [500, 200],
    [200, 500]
]

t_obs = np.linspace(0, 1000, 1000)  # Observation times

# Plotting
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(1, 1, 1)

for s0 in s0_vals:
    s_obs = odeint(sdot3, s0, t_obs, args=(params,))
    PL_obs = s_obs[:, 0]
    PT_obs = s_obs[:, 1]
    ax.plot(PL_obs, PT_obs, label=f'Initial PL={s0[0]}, PT={s0[1]}')

# Nullclines
PT_vals = np.linspace(0, 3000, 300)
PL_nullcline = [((vp0L / vdpL) + (vpL / vdpL) * (KT**nT / (pt**nT + KT**nT))) for pt in PT_vals]
PT_nullcline = [((vp0T / vdpT) + (vpT / vdpT) * (KL**nL / (pl**nL + KL**nL))) for pl in PL_nullcline]

ax.plot(PL_nullcline, PT_vals, 'r--', label='PL Nullcline')
ax.plot(PT_nullcline, PL_nullcline, 'b--', label='PT Nullcline')

ax.set_xlim([0, 3000])
ax.set_ylim([0, 3000])
ax.set_xlabel('LacI Protein (PL)')
ax.set_ylabel('TetR Protein (PT)')
ax.set_title('Phase Plot of Toggle Switch System with n=1')
ax.legend()

plt.show()


3i)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

def sdot(s, t, params):
    ML, PL, MT, PT = s
    km0L, kmL, KT, nT, kdmL, kpL, kdpL, km0T, kmT, KL, nL, kdmT, kpT, kdpT = params

    rate_ML_prod = km0L + kmL * (KT**nT / (PT**nT + KT**nT))
    rate_ML_loss = kdmL * ML

    rate_PL_prod = kpL * ML
    rate_PL_loss = kdpL * PL

    rate_MT_prod = km0T + kmT * (KL**nL / (PL**nL + KL**nL))
    rate_MT_loss = kdmT * MT

    rate_PT_prod = kpT * MT
    rate_PT_loss = kdpT * PT
    
    dML = rate_ML_prod - rate_ML_loss
    dPL = rate_PL_prod - rate_PL_loss
    dMT = rate_MT_prod - rate_MT_loss
    dPT = rate_PT_prod - rate_PT_loss
    
    return [dML, dPL, dMT, dPT]

# Parameter values from Lugange et al.
params = [
    0.032, 8.30, 30, 2, 0.1386, 0.9726, 0.0165, # LacI parameters
    0.119, 2.06, 31.94, 2, 0.1386, 1.170, 0.0165  # TetR parameters
]

# Initial conditions
initial_conditions = [
    [10, 500, 10, 10],   # Close to one stable state
    [10, 10, 10, 500]    # Close to another stable state
]

t_obs = np.linspace(0, 1000, 1000)  # Observation times

# Simulate and plot
fig, axs = plt.subplots(2, 2, figsize=(12, 12))

for i, s0 in enumerate(initial_conditions):
    s_obs = odeint(sdot, s0, t_obs, args=(params,))
    ML_obs, PL_obs, MT_obs, PT_obs = s_obs[:, 0], s_obs[:, 1], s_obs[:, 2], s_obs[:, 3]

    # Expression vs Time plot
    axs[i, 0].plot(t_obs, PL_obs, label='LacI Protein')
    axs[i, 0].plot(t_obs, PT_obs, label='TetR Protein')
    axs[i, 0].set_xlabel('Time')
    axs[i, 0].set_ylabel('Protein count')
    axs[i, 0].set_title(f'Expression Over Time (Initial PL={s0[1]}, PT={s0[3]})')
    axs[i, 0].legend()

    # Phase plot
    axs[i, 1].plot(PL_obs, PT_obs)
    axs[i, 1].set_xlabel('LacI Protein')
    axs[i, 1].set_ylabel('TetR Protein')
    axs[i, 1].set_title('Phase Plot')
    axs[i, 1].set_xlim([0, 1000])
    axs[i, 1].set_ylim([0, 1000])

plt.tight_layout()
plt.show()


3ii)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random

# Gillespie simulation function
def gillespie_toggle(s0, t_obs_out, param):
    km0L, kmL, KL, nL, kdmL, kpL, kdpL, km0T, kmT, KT, nT, kdmT, kpT, kdpT = param
    ML, PL, MT, PT = s0

    s_obs = [s0]
    t_obs = [t_obs_out[0]]
    t = t_obs_out[0]

    while t < t_obs_out[-1]:
        # Event types
        types = ['ML_prod', 'ML_deg', 'PL_prod', 'PL_deg', 'MT_prod', 'MT_deg', 'PT_prod', 'PT_deg']

        # Calculate reaction rates
        rate_ML_prod = km0L + kmL * (KT**nT / (PT**nT + KT**nT))
        rate_ML_deg = kdmL * ML
        rate_PL_prod = kpL * ML
        rate_PL_deg = kdpL * PL
        rate_MT_prod = km0T + kmT * (KL**nL / (PL**nL + KL**nL))
        rate_MT_deg = kdmT * MT
        rate_PT_prod = kpT * MT
        rate_PT_deg = kdpT * PT

        rates = [rate_ML_prod, rate_ML_deg, rate_PL_prod, rate_PL_deg, rate_MT_prod, rate_MT_deg, rate_PT_prod, rate_PT_deg]

        total_rate = sum(rates)
        if total_rate == 0:
            break

        # Time to next event
        next_event_time = gen_next_event_time(total_rate)
        t += next_event_time

        # Determine which event occurs
        event_index = random_choice_from_pdf([rate / total_rate for rate in rates])
        event_type = types[event_index]

        # Update counts based on event
        if event_type == 'ML_prod':
            ML += 1
        elif event_type == 'ML_deg':
            ML = max(ML - 1, 0)
        elif event_type == 'PL_prod':
            PL += 1
        elif event_type == 'PL_deg':
            PL = max(PL - 1, 0)
        elif event_type == 'MT_prod':
            MT += 1
        elif event_type == 'MT_deg':
            MT = max(MT - 1, 0)
        elif event_type == 'PT_prod':
            PT += 1
        elif event_type == 'PT_deg':
            PT = max(PT - 1, 0)

        s_obs.append((ML, PL, MT, PT))
        t_obs.append(t)

    s_obs_out = resample_observations(t_obs, s_obs, t_obs_out)
    return np.array(s_obs_out)

# Parameters for the Toggle Switch
params = [0.032, 8.3, 30, 2, 0.1386, 0.9726, 0.0165, 0.119, 2.06, 31.94, 2, 0.1386, 1.170, 0.0165]

# Initial conditions for LacI high and TetR high
s0_LacI_high = [0, 2600, 0, 100]
s0_TetR_high = [0, 100, 0, 2600]

# Define observation times
t_obs = np.linspace(0, 1000, 1000)

# Run multiple simulations for each initial condition
results_LacI_high = [gillespie_toggle(s0_LacI_high, t_obs, params) for _ in range(5)]
results_TetR_high = [gillespie_toggle(s0_TetR_high, t_obs, params) for _ in range(5)]

# Plot results for LacI high initial state
plt.figure(figsize=(12, 6))
for result in results_LacI_high:
    plt.plot(t_obs, result[:, 1], label='LacI Protein')
    plt.plot(t_obs, result[:, 3], label='TetR Protein', alpha=0.5)
plt.title('Toggle Switch Dynamics (LacI High Initial State)')
plt.xlabel('Time')
plt.ylabel('Protein Count')
plt.legend()
plt.show()

# Plot results for TetR high initial state
plt.figure(figsize=(12, 6))
for result in results_TetR_high:
    plt.plot(t_obs, result[:, 1], label='LacI Protein', alpha=0.5)
    plt.plot(t_obs, result[:, 3], label='TetR Protein')
plt.title('Toggle Switch Dynamics (TetR High Initial State)')
plt.xlabel('Time')
plt.ylabel('Protein Count')
plt.legend()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random

# Parameters for the Toggle Switch
params = [0.032, 8.3, 30, 2, 0.1386, 0.9726, 0.0165, 0.119, 2.06, 31.94, 2, 0.1386, 1.170, 0.0165]

# Initial conditions
s0_LacI_high = [0, 2600, 0, 100]  # ML0, PL0, MT0, PT0 (High LacI state)
s0_TetR_high = [0, 100, 0, 2600]  # ML0, PL0, MT0, PT0 (High TetR state)

# Observation time
t_obs = np.linspace(0, 1000, 1001)

# Run simulations
num_runs = 100
results_LacI_high = [gillespie_toggle(s0_LacI_high, t_obs, params) for _ in range(num_runs)]
results_TetR_high = [gillespie_toggle(s0_TetR_high, t_obs, params) for _ in range(num_runs)]

# Collect final mRNA and protein levels
final_ML_LacI = [res[-1, 0] for res in results_LacI_high]
final_PL_LacI = [res[-1, 1] for res in results_LacI_high]
final_MT_LacI = [res[-1, 2] for res in results_LacI_high]
final_PT_LacI = [res[-1, 3] for res in results_LacI_high]

final_ML_TetR = [res[-1, 0] for res in results_TetR_high]
final_PL_TetR = [res[-1, 1] for res in results_TetR_high]
final_MT_TetR = [res[-1, 2] for res in results_TetR_high]
final_PT_TetR = [res[-1, 3] for res in results_TetR_high]

# Calculate summary statistics
mean_PL_LacI = np.mean(final_PL_LacI)
std_PL_LacI = np.std(final_PL_LacI)
mean_PT_LacI = np.mean(final_PT_LacI)
std_PT_LacI = np.std(final_PT_LacI)

mean_PL_TetR = np.mean(final_PL_TetR)
std_PL_TetR = np.std(final_PL_TetR)
mean_PT_TetR = np.mean(final_PT_TetR)
std_PT_TetR = np.std(final_PT_TetR)

# Plot histograms
plt.figure(figsize=(12, 10))

plt.subplot(2, 2, 1)
plt.hist(final_PL_LacI, bins=20, alpha=0.7, color='blue')
plt.title('LacI Protein Levels (High LacI Initial)')
plt.xlabel('Protein Count')
plt.ylabel('Frequency')
plt.axvline(mean_PL_LacI, color='r', linestyle='dashed', linewidth=1)
plt.text(mean_PL_LacI + std_PL_LacI, max(plt.ylim())*0.9, f'Mean: {mean_PL_LacI:.2f}\nSD: {std_PL_LacI:.2f}')

plt.subplot(2, 2, 2)
plt.hist(final_PT_LacI, bins=20, alpha=0.7, color='green')
plt.title('TetR Protein Levels (High LacI Initial)')
plt.xlabel('Protein Count')
plt.ylabel('Frequency')
plt.axvline(mean_PT_LacI, color='r', linestyle='dashed', linewidth=1)
plt.text(mean_PT_LacI + std_PT_LacI, max(plt.ylim())*0.9, f'Mean: {mean_PT_LacI:.2f}\nSD: {std_PT_LacI:.2f}')

plt.subplot(2, 2, 3)
plt.hist(final_PL_TetR, bins=20, alpha=0.7, color='blue')
plt.title('LacI Protein Levels (High TetR Initial)')
plt.xlabel('Protein Count')
plt.ylabel('Frequency')
plt.axvline(mean_PL_TetR, color='r', linestyle='dashed', linewidth=1)
plt.text(mean_PL_TetR + std_PL_TetR, max(plt.ylim())*0.9, f'Mean: {mean_PL_TetR:.2f}\nSD: {std_PL_TetR:.2f}')

plt.subplot(2, 2, 4)
plt.hist(final_PT_TetR, bins=20, alpha=0.7, color='green')
plt.title('TetR Protein Levels (High TetR Initial)')
plt.xlabel('Protein Count')
plt.ylabel('Frequency')
plt.axvline(mean_PT_TetR, color='r', linestyle='dashed', linewidth=1)
plt.text(mean_PT_TetR + std_PT_TetR, max(plt.ylim())*0.9, f'Mean: {mean_PT_TetR:.2f}\nSD: {std_PT_TetR:.2f}')

plt.tight_layout()
plt.show()
